In [1]:
import pandas as pd

In [2]:
%pwd

'/Users/a06411/Documents/GitHub/js_automl_pycaret'

In [3]:
path = "/Users/a06411/Documents/data_hub/lending_club/lending_club.csv"

In [4]:
# 데이터 불러오기
data = pd.read_csv(path)

/var/folders/h4/44486vyn1_xbs13g1z50l5m00000gn/T/ipykernel_17623/8498173.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [5]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,68407277,NaN,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,68355089,NaN,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,68341763,NaN,20000.0,20000.0,20000.0,60 months,10.78,432.66,B,B4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,66310712,NaN,35000.0,35000.0,35000.0,60 months,14.85,829.90,C,C5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,68476807,NaN,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
data.shape

(2260701, 151)

In [16]:
import pyspark

In [17]:
from pyspark.sql import SparkSession

# Create SparkSession 
spark = (SparkSession.builder 
      .config("spark.driver.host","127.0.0.1") 
      .config("spark.driver.bindAddress","127.0.0.1")
      .appName("SparkByExamples.com") 
      .getOrCreate() )

In [18]:
data = spark.read.csv(path, header=True, inferSchema=True)

In [21]:
# 데이터를 80메가 크기로 분할
repartitioned_data = data.repartition(26)  # 26개의 파티션으로 분할 (2GB / 80MB = 26)


In [22]:
# 분할된 데이터를 저장
repartitioned_data.write.format("parquet").mode("overwrite").save(f"./data/lending_club_parquet/")

23/09/08 13:01:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/09/08 13:01:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/09/08 13:01:15 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 76.00% for 10 writers
23/09/08 13:01:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 84.44% for 9 writers
23/09/08 13:01:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/09/08 13:01:19 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
23/09/08 13:01:19 WARN MemoryManager: Total allocation exceeds 95.00%

In [23]:
parquet_data = spark.read.parquet("./data/lending_club_parquet")

In [24]:
# 데이터 건수 확인
count = parquet_data.count()
print("데이터 건수:", count)

데이터 건수: 2260701


In [27]:
# 파케이 파일 읽기
parquet_data = pd.read_parquet("./data/lending_club_parquet/part-00000-82598917-8601-4344-bce8-73f14b90ecef-c000.snappy.parquet")

In [28]:
parquet_data.shape

(86950, 151)